# Artist Plays
> Tracking artist plays over time.

- toc: true 
- badges: true
- comments: false
- categories: [asot, bpm]
- image: images/artist-plays.png

In [2]:
#hide
import os
import yaml
import spotipy
import json
import altair as alt
import numpy as np
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

with open('spotipy_credentials.yaml', 'r') as spotipy_credentials_file:
    credentials = yaml.safe_load(spotipy_credentials_file)
    os.environ["SPOTIPY_CLIENT_ID"] = credentials['spotipy_credentials']['spotipy_client_id']
    os.environ["SPOTIPY_CLIENT_SECRET"] = credentials['spotipy_credentials']['spotipi_client_seret']

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

asot_radio_id = '25mFVpuABa9GkGcj9eOPce'

albums = []
results = sp.artist_albums(asot_radio_id, album_type='album')
albums.extend(results['items'])
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])
seen = set()  # to avoid dups
for album in albums:
    name = album['name']
    if name not in seen:
        seen.add(name)

singles = []
results = sp.artist_albums(asot_radio_id, album_type='single')
singles.extend(results['items'])
while results['next']:
    results = sp.next(results)
    singles.extend(results['items'])
seen = set()  # to avoid dups
for single in singles:
    name = single['name']
    if name not in seen:
        seen.add(name)

episodes = singles + albums

episodes.sort(key=lambda x: x['release_date']) # Sort by release date

## Introduction

How many unique artists have been played on A State of Trance over the years? Who's been played the most? And the least?

In this post, we'll examine artist plays over time.

## Getting Started

First, some baseline analysis. Let's first figure out how many tracks played on A State of Trance are available on Spotify:

In [3]:
tracks_counted = 0

for episode in episodes:
    try:
        for track in sp.album_tracks(episode['uri'])['items']:
            if "a state of trance" in track['name'].lower() or "- interview" in track['name'].lower():
                continue
            else:
                tracks_counted += 1
    except:
        pass

print(tracks_counted)

17311


Wow - 17,000+ total _(not unique)_ tracks have been played! Remember, as we learned in the ["Methodology" post](https://scottbrenner.github.io/asot-jupyter/asot/bpm/2020/04/27/methodology.html) some episodes - especially early ones - are incomplete.

How many unique artists?

In [9]:
unique_artists = set()

for episode in episodes:
    try:
        for track in sp.album_tracks(episode['uri'])['items']:
            if "a state of trance" in track['name'].lower() or "- interview" in track['name'].lower():
                continue
            else:
                for artist in track['artists']:
                    unique_artists.add(artist['name'])
    except:
        pass

print(len(unique_artists))

4079


As always, this is a "best guess" - an approximation.

For the sake of tallying unique artists, we are treating collaborators as individuals. A track produced by "Artist A & Artist B" is recorded here as a production by Artist A and Artist B invididually.

## Calculating

Let's crunch some numbers.

Which artists have the most plays?

In [17]:
from collections import defaultdict

artist_counter = defaultdict(int)

for episode in episodes:
    try:
        for track in sp.album_tracks(episode['uri'])['items']:
            if "a state of trance" in track['name'].lower() or "- interview" in track['name'].lower():
                continue
            else:
                for artist in track['artists']:
                    artist_counter[artist['name']] += 1
    except:
        pass

top_artists = sorted(artist_counter.items(), key=lambda k_v: k_v[1], reverse=True)

Alright, let's see the top 25 in a graph..

In [32]:
source = pd.DataFrame.from_dict(top_artists[:25])

bars = alt.Chart(source).mark_bar().encode(
    x=alt.X('1:Q', title='Plays'),
    y=alt.Y('0:N', sort='-x', title='Artist')
).properties(
    title="A State of Trance - Most-played artists",
    width=600
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='1:Q'
)

bars + text

alt.LayerChart(...)

First place goes to the big man himself of course. Keep in mind we're counting a remix of an artist's track as a play for that artist.

From our numbers, a track that credits Armin van Buuren has been played 1210 times across 960 episodes. From this, we can say the average episode of A State of Trance has

In [24]:
top_artists[0][1] / len(episodes)

1.2604166666666667

tracks produced by Armin van Buuren in some form or another, which is totally useless to know.

Earlier we found 17,000+ total tracks played from 4000+ unique artists based on what's currently available on Spotify. How many artists have been played exactly once?

In [29]:
artists_played_once = 0
one_hit_wonders = []

for artist, plays in artist_counter.items():
    if plays == 1:
        one_hit_wonders.append(artist)
        artists_played_once += 1

print(artists_played_once)

1769


Seems like quite a bit, what percentage is that?

In [25]:
print(100 * artists_played_once / len(unique_artists))

43.36847266486884


From the data available on Spotify, we can say 43% of artists played on A State of Trance were played exactly once. A full list of artists played once appears below, collapsed:

In [31]:
#collapse
print(sorted(one_hit_wonders))

['16BL', '2Symmetry', '3 Amigos', '3 Colours', '3 Of Life', '4 Mal', '4Mal', '7 Baltic', '8.02', 'A Lost Connection', 'A&Z', 'AA Meeting', 'ABHISHEK Y2V', 'AMTM', 'ANDO', 'ANIMA!', 'ANUQRAM', 'ANVY', 'ASCO', 'ATHOM', 'ATN', 'AWD', 'Abaze', 'Abraham Leoga', 'Ace Ventura', 'Active State', 'Adagio Sensus', 'Adaja', 'Adam Coppack', 'Adam Deane', 'Adam K & Soha', 'Adam Lambert', 'Adam Lavel', 'Adam Lindburg', 'Adam Sheridan', 'Adam Sky', 'Adam Sobiech', 'Adam Tas', 'Adiva', 'Adrian Blue', 'Adrian Font', 'Adrian&Raz', 'Adriana Thorpe', 'Advanced', 'Adymus', 'Aero 21', 'Aether', 'After Midnight', 'AfterU', 'Against The Grain', 'Aguada', 'Ahmed Atef', 'Ahorn', 'Aiden Arie', 'Aiera & Osiris', 'Aimo', 'AirLab7', 'Airsun', 'Airtight', 'Airway 7', 'Aisling Jarvis', 'Aki Bergen & Richter', 'Akku', 'Aknael & Bekeela', 'Aknow', 'Alae Khaldi', 'Alan Banks', 'Alan Stott', 'Alan Wyse', 'Albert Keyn', 'Albion', 'Alco Rhyhtm', 'Aldo Henrycho', 'Aleksey Gunichev', 'Aleksey Sladkov', 'Alex', 'Alex Arestegui

Highlights include Steve Aoki and What So Not, among others. Again, this is based on how artists are reported in Spotify and as such is not entirely accurate. In the above we see 'tAudrey Gallagher' and "Thomas Datt's", both typos, report as separate from their namesake.

## Summary

In conclusion, across the current (as of writing) 960 episodes of A State of Trance...
* 17,000+ total tracks have been played
* 4,000+ unique artists have been featured
* 1,200+ tracks produced by Armin van Buuren in some form or another have been played
* 1,700+ artists (43%) have been played exactly once

And there we have it! There's plenty more to investigate, stay tuned..